# PyTorch tutorials quickstart
web recources at  [Quickstart](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)

## Working with data
PyTorch has two primitives to work with data: ```torch.utils.data.DataLoader``` and ```torch.utils.data.Dataset```. ```Dataset``` stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

## Download and store datasets

we use the FashionMNIST dataset. Every ```TorchVision Dataset``` includes two arguments: ```transform``` and ```target_transform``` to modify the samples and labels respectively.

In [2]:
# using FashionMNIST

# Download training data from open datasets.
# data will be download to "/Users/eliascheung(YOUR NAME)/Data" by default, you can modify it by 
# change the param : root, let's change it to "./data" --> same directory as this notebook

from pathlib import Path

working_dir = Path().cwd()
print(working_dir)
data_dir = working_dir / Path("data") # ./data

training_data = datasets.FashionMNIST(
    root=data_dir,
    train=True, # True for train and False for test
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root=data_dir,
    train=False, # True for train and False for test
    download=True,
    transform=ToTensor(),
)

/Users/eliascheung/Documents/LEARNING/pytorch_learning


## Pass dataset to DataLoader

this operation wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading. 

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

# N: means batch size: 
# C: channel counts of an image 
# H: height of an image
# W: width of an image

# ToTensor():
# Converts a PIL Image or numpy.ndarray (H x W x C) in the range [0, 255] to a 
# torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0] if the PIL Image 
# belongs to one of the modes (L, LA, P, I, F, RGB, YCbCr, RGBA, CMYK, 1) or 
# if the numpy.ndarray has dtype = np.uint8
#In the other cases, tensors are returned without scaling
#

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Creating Models
To define a neural network in PyTorch, we create a class that inherits from ```nn.Module```. We define the layers of the network in the ```__init__``` function and specify how data will pass through the network in the ```forward``` function. 

In [4]:
# specify device to run/accelerate models
device = "mps" if torch.backends.mps.is_available() \
               else "cuda" if torch.cuda.is_available() \
               else "cpu"
print(f"{device} if available")

mps if available


detailed doc seen at [Build the Neural Network](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)

In [5]:
# Define model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):  
        # Do not call model.forward() directly!
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# instantiate a model and move it to GPU if available
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
print(28 * 28)

784


## Training（Optimizing） the Model Parameters

To train a model, we need a ```loss function``` [🔍](https://pytorch.org/docs/stable/nn.html#loss-functions) and an ```optimizer```[🔍](https://pytorch.org/docs/stable/optim.html)



[nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss)

[torch.optim.SGD](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD)

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

in a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model’s parameters.

In [8]:
# train_dataloader = DataLoader(training_data, batch_size=batch_size)
# test_dataloader = DataLoader(test_data, batch_size=batch_size)
# train_dataloader and test_dataloader ---> dataloader params in train function and 
# test function

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # batch: index of (X, y) of dataloader(train_dataloader)
        # X: train data tensors, y: label of X
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

check the model’s performance against the test dataset to ensure it is learning.

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (epochs). During each epoch, the model learns parameters to make better predictions. We print the model’s accuracy and loss at each epoch; we’d like to see the accuracy increase and the loss decrease with every epoch.

In [10]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.294050  [   64/60000]
loss: 2.282475  [ 6464/60000]
loss: 2.266765  [12864/60000]
loss: 2.263417  [19264/60000]
loss: 2.243523  [25664/60000]
loss: 2.209604  [32064/60000]
loss: 2.216442  [38464/60000]
loss: 2.179053  [44864/60000]
loss: 2.174889  [51264/60000]
loss: 2.136566  [57664/60000]
Test Error: 
 Accuracy: 46.1%, Avg loss: 2.133237 

Epoch 2
-------------------------------
loss: 2.141585  [   64/60000]
loss: 2.135608  [ 6464/60000]
loss: 2.068391  [12864/60000]
loss: 2.088719  [19264/60000]
loss: 2.034062  [25664/60000]
loss: 1.962537  [32064/60000]
loss: 1.990236  [38464/60000]
loss: 1.897013  [44864/60000]
loss: 1.907006  [51264/60000]
loss: 1.829053  [57664/60000]
Test Error: 
 Accuracy: 56.4%, Avg loss: 1.827939 

Epoch 3
-------------------------------
loss: 1.861624  [   64/60000]
loss: 1.837426  [ 6464/60000]
loss: 1.702784  [12864/60000]
loss: 1.750888  [19264/60000]
loss: 1.645854  [25664/60000]
loss: 1.589526  [32064/600

## Saving Models
A common way to save a model is to serialize the internal state dictionary (containing the model parameters).

In [11]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## Loading Models
The process for loading a model includes re-creating the model structure and loading the state dictionary into it.

In [12]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

## Prediction

In [13]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
